**LIBRERIAS**

In [151]:
#Antes que nada es muy importante descargar 'vader_lexico' ya que sin este recurso no podremos utilizar la funcion 'SentimentIntensityAnalyzer(). 'Vader_lexicon' se utiliza en el analisis de sentimiento para determinar la polaridad de un texto de una frase. Es una lista de palabras en ingles estan asociados con valores de polaridad y sentimiento.
#También debemos importar pandas
import nltk
nltk.download('vader_lexicon')
import pandas as pd
from googletrans import Translator

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
#Una vez descargado el 'vader_lexicon', podremos usar de manera correcta la funcion de 'SentimentIntensityAnalyzer()'
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
#asignamos a una varible la función importada
sia = SentimentIntensityAnalyzer()

**IMPORTAMOS EL ARCHIVO AL QUE LE QUEREMOS REALIZAR EL 'SENTIMENT ANALYSIS**

In [148]:
#Abrimos el archivo
df_reviews = pd.read_csv('reviews_v2.0_fechas_limpias.csv')

In [29]:
df_reviews.head()

,date,year,item_id,user_id,recommend,review
0,2011-11-05,2011,1250,76561197970982479,True,Simple yet with great replayability. In my opi...
1,2011-07-15,2011,22200,76561197970982479,True,It's unique and worth a playthrough.
2,2011-04-21,2011,43110,76561197970982479,True,Great atmosphere. The gunplay can be a bit chu...
3,2014-06-24,2014,251610,js41637,True,I know what you think when you see this title ...
4,2013-09-08,2013,227300,js41637,True,For a simple (it's actually not all that simpl...


In [ ]:
#hacemos una copia para trabajar
df_reviews_2 = df_reviews.copy()

In [40]:
df_reviews_2.head()

,date,year,item_id,user_id,recommend,review,sentiment_analysis,sentiment_analysis_textblob
0,2011-11-05,2011,1250,76561197970982479,True,Simple yet with great replayability. In my opi...,0.8481,0.174444
1,2011-07-15,2011,22200,76561197970982479,True,It's unique and worth a playthrough.,0.2263,0.337500
2,2011-04-21,2011,43110,76561197970982479,True,Great atmosphere. The gunplay can be a bit chu...,0.9117,0.050000
3,2014-06-24,2014,251610,js41637,True,I know what you think when you see this title ...,0.9566,0.114583
4,2013-09-08,2013,227300,js41637,True,For a simple (it's actually not all that simpl...,0.9708,-0.036111


In [111]:
#Para las filas de la columna reviews que no tengan nada lo vamos a rellenar con neutre, para que el analisis lea el mensaje neutre me retorne 1, ya que la consigna lo solicita
df_reviews_2['review'] = df_reviews_2['review'].fillna('neutre')

**HACEMOS EL ANALISIS DE SENTIMIENTO**

In [32]:
#Pasamos todos los valores a str
df_reviews_2['review'] = df_reviews_2['review'].astype(str)

In [34]:
#Aplicamos el sentiment analysis y tomamos el compound que es como una media de si el valor es nagativo, neutro, positivo
df_reviews_2['sentiment_analysis'] = df_reviews_2['review'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [186]:
#Definimos la funcion que va determinar los valores de negativo, neutral, positivo
def sentiment_analysis(review):
    try:
        # Analizar el sentimiento en el texto traducido
        analysis = sia.polarity_scores(review)

        if analysis['compound'] > -0.94 and analysis['compound'] < 0.24: #el neutral en compound es 0, le damos más margen de error
            return 1  # Neutral
        elif analysis['compound'] >= 0.24 or analysis['pos'] > 0.48: #si es mayor o igual a 0.24 lo consideramos positivo, mientras compound este mas cerca de 1 es positivo, o sino lo más directo es ver 'pos'
            return 2  # Positivo
        else:
            return 0  # Negativo
    except Exception as e:
        print(f"Error en el análisis de sentimiento: {str(e)}")
        return 1  # Neutral (si ocurre un error)


In [187]:
#aplicamos la funcion al review
df_reviews_2['sentiment_analysis'] = df_reviews_2['review'].apply(sentiment_analysis)

In [188]:
df_reviews_2

,date,year,item_id,user_id,recommend,review,sentiment_analysis
0,2011-11-05,2011,1250,76561197970982479,True,Simple yet with great replayability. In my opi...,2
1,2011-07-15,2011,22200,76561197970982479,True,It's unique and worth a playthrough.,1
2,2011-04-21,2011,43110,76561197970982479,True,Great atmosphere. The gunplay can be a bit chu...,2
3,2014-06-24,2014,251610,js41637,True,I know what you think when you see this title ...,2
4,2013-09-08,2013,227300,js41637,True,For a simple (it's actually not all that simpl...,2
...,...,...,...,...,...,...,...
59300,2014-07-10,2014,70,76561198312638244,True,a must have classic from steam definitely wort...,2
59301,2014-07-08,2014,362890,76561198312638244,True,this game is a perfect remake of the original ...,2
59302,2014-07-03,2014,273110,LydiaMorley,True,had so much fun plaing this and collecting res...,2
59303,2014-07-20,2014,730,LydiaMorley,True,:D,2


**AHORA VAMOS A ORDENAR Y ELIMINAR COLUMNAS DEL DF**

In [189]:
df_reviews_2 = df_reviews_2[['item_id', 'user_id', 'year', 'recommend', 'sentiment_analysis']]

In [191]:
df_reviews_2.head()

,item_id,user_id,year,recommend,sentiment_analysis
0,1250,76561197970982479,2011,True,2
1,22200,76561197970982479,2011,True,1
2,43110,76561197970982479,2011,True,2
3,251610,js41637,2014,True,2
4,227300,js41637,2013,True,2


**EXPORTAMOS**

In [192]:
df_reviews_2.to_csv('reviews_v3.0_con_analisis_sentimiento.csv', index=False)